<a href="https://colab.research.google.com/github/yutawatabe/quantiative_trade_lecture/blob/main/Computing_Eaton_Kortum_model_full_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **パラメーターから均衡へ**

シミュレートした適当なパラメーターから均衡の計算を行う。

In [132]:
# numpy, mathをインポート
import numpy as np # 行列計算のライブラリ
from math import gamma # 価格指数のため

## まずパラメーターを設定する。

In [133]:
N = 3 # 国の数
theta = 4 # 貿易弾力性のパラメーター
sigma = 3 # 代替の弾力性のパラメーター

In [134]:
T = np.array([1., 1., 1.]) # 技術パラメーター
print(T)
print(T.shape)

[1. 1. 1.]
(3,)


In [135]:
L = np.array([1,1.5,1.5]) # 人口
print(L)
print(L.shape)

[1.  1.5 1.5]
(3,)


まず $\boldsymbol{\tau}$のすべての要素を1.5として、そのあとにループを使用して国内貿易費用を1とする。

In [136]:
tau = np.ones((N,N)) * 1.5 # 貿易費用
print(tau)
print(tau.shape)
for OR in np.ndindex((N)):
  print(OR)
  tau[OR,OR] = 1 #国内貿易費用は1

print(tau)

[[1.5 1.5 1.5]
 [1.5 1.5 1.5]
 [1.5 1.5 1.5]]
(3, 3)
(0,)
(1,)
(2,)
[[1.  1.5 1.5]
 [1.5 1.  1.5]
 [1.5 1.5 1. ]]


**仮置きした賃金での均衡の確認**

仮に国の賃金が同一として、果たして労働市場（財市場）が均衡するかを見ていく。候補として全ての国 $i$ に対して
$$ w_i = 1 $$
を考えてみる。


In [98]:
w = np.ones((N))
print(w)

[1. 1. 1.]


まず総所得（消費）を計算しておく。$X_n$を総消費とする。この経済では労働のみが生産要素なので、
$$X_n = w_n L_n$$
となる。

In [99]:
Xn = w * L
print(Xn)

[1.  1.5 1.5]


まず投入費用$c_{in}$を計算する
$$ c_{in} = w_i \tau_{in} $$

In [100]:
c = np.zeros((N,N)) # i国からn国までの投入費用（を入れるための箱）
for OR,DE in np.ndindex((N,N)):
  print(OR)
  print(DE)
  c[OR,DE] = w[OR] * tau[OR,DE]
  print(c[OR,DE])


0
0
1.0
0
1
1.5
0
2
1.5
1
0
1.5
1
1
1.0
1
2
1.5
2
0
1.5
2
1
1.5
2
2
1.0


それを使って輸入シェアを計算する。
$$ \pi_{in} = \frac{ T_{i} c_{in}^{-\theta}}{\sum_{k=1} T_{k} c_{ki}^{-\theta} }   $$
コーディングの際には以下のような表記を用いる。
$$ \pi_{in} = \frac{\pi_{in,num}}{\Phi_{n}} $$
ここでは
$$ \pi_{in,num} = T_{i} c_{in}^{-\theta} $$
であり、
$$ \Phi_{n} = \sum_{k=1}^N T_{i} c_{kn}^{-\theta} $$
となっている。


In [123]:
pi = np.zeros((N,N)) # 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）
pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
Phi = np.zeros((N)) # 輸入シェアの分母部分

for OR,DE in np.ndindex((N,N)):
  pi_num[OR,DE] = T[OR] * (w[OR] * tau[OR,DE]) ** (-theta)
  Phi[DE] += pi_num[OR,DE] # pi_den[DE] = pi_den[DE] + pi_num[OR,DE]

for OR,DE in np.ndindex((N,N)):
  pi[OR,DE] = pi_num[OR,DE] / Phi[DE]

print(pi)

[[0.66937672 0.11655476 0.11655476]
 [0.16531164 0.73772231 0.14572293]
 [0.16531164 0.14572293 0.73772231]]


ここで、$\pi_{in}$の定義上、$ \sum_{i=1}^N \pi_{in} = 1 $ が成立するはず。
また仮置きした賃金が同一であり、国の技術、貿易費用が対象のため、$\boldsymbol{\pi}$も対称になる。それを上で確認する。

ここから価格指数を計算しておく（あとで厚生の評価に使う）
$$ P_{n} = \Gamma \left( \frac{\theta + \sigma -1}{\theta} \right) \Phi_{n}^{-1/\theta} $$

In [125]:
P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)
print(P)

[0.25040568 0.24263323 0.24263323]


$i$ 国から $n$ 国への輸出は、輸入シェアに輸入国の総消費をかけたものになる。
$$X_{in} = \pi_{in} X_n $$

In [102]:
X = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  X[OR,DE] = pi[OR,DE] * Xn[DE]

print(X)

[[0.71681416 0.21238938 0.21238938]
 [0.14159292 1.07522124 0.21238938]
 [0.14159292 0.21238938 1.07522124]]


ここでは$\boldsymbol{\pi}$のように足して1にならず、対称でもない。これは需要規模が国によって違うため（消費者の数が違う）。

**労働市場の均衡の確認**

まず労働供給は固定であり、$\boldsymbol{L} = \boldsymbol{L}_S$になる。

In [103]:
L_S = L
print(L)

[1.  1.5 1.5]


労働需要は、総売上を賃金で割ると導ける。数学的には
$$ L_{i,D} = \frac{ X_{in}}{w_i} $$
が労働需要になる。これを計算して、労働の超過需要をZを求めてあげる。
$$ Z_{i} = L_{i,D} - L_i $$

In [104]:
L_D = np.zeros((N))
for OR,DE in np.ndindex((N,N)):
  L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]

Z = L_D - L_S

この賃金が労働市場を均衡させる賃金なら、Zはどの国でも0になるはず。

In [105]:
print(Z)

[ 0.14159292 -0.07079646 -0.07079646]


ならない、ということはこの賃金は均衡賃金ではない！
次の賃金候補を探してあげたい。今の賃金を $w^0_i$とし（最初のinterationの賃金）、アップデート方法を考える。ここでは超過需要を用いたアップデータ方を用いる。

In [106]:
psi = 0.1 # 収束のスピードをコントロールするパラメーター

w_new = w * (1 + psi * (Z / L) )

ここで新しい$w$と古い$w$を比較する。ちゃんと超過需要が発生していた国において賃金が高くなっているか確認できるはず。

In [107]:
print(Z)
print(w_new - w)

[ 0.14159292 -0.07079646 -0.07079646]
[ 0.01415929 -0.00471976 -0.00471976]


均衡賃金は基準化なしに一意にはきまらない。最後に世界のGDPが1になるように基準化しておく。

In [108]:
wgdp = np.sum(w_new * L)
print(wgdp)
w_new = w_new / wgdp
print(w_new)

4.0
[0.25353982 0.24882006 0.24882006]


いままでの流れをまとめると


1.   パラメーターを設定する
2.   仮定した賃金をもとに、輸入シェア、貿易額を計算する
3.   貿易額を用いて、労働市場の均衡をチェックする
4.   労働の超過需要を用いて、仮置きした賃金を更新していく。

この2から4を**関数**にまとめる。


In [126]:
def updatewage(w,theta,sigma,N,L,T,tau,psi):
  Xn = w * L

  c = np.zeros((N,N)) # i国からn国までの投入費用（を入れるための箱）
  for OR,DE in np.ndindex((N,N)):
    c[OR,DE] = w[OR] * tau[OR,DE]

  pi = np.zeros((N,N)) # 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）
  pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
  pi_den = np.zeros((N)) # 輸入シェアの分母部分

  for OR,DE in np.ndindex((N,N)):
    pi_num[OR,DE] = T[OR] * (w[OR] * tau[OR,DE]) ** (-theta)
    pi_den[DE] += pi_num[OR,DE] # pi_den[DE] = pi_den[DE] + pi_num[OR,DE]

  for OR,DE in np.ndindex((N,N)):
    pi[OR,DE] = pi_num[OR,DE] / pi_den[DE]

  P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)

  X = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    X[OR,DE] = pi[OR,DE] * Xn[DE]

  L_S = L
  L_D = np.zeros((N))
  for OR,DE in np.ndindex((N,N)):
    L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]

  Z = L_D - L_S
  w_new = w * (1 + psi * (Z / L) )

  wgdp = np.sum(w_new * L)
  w_new = w_new / wgdp

  return w_new,Z,P,X

ここでちゃんと結果が前の関数なしで書いたiterationと一致するかを確認する

In [110]:
w_newfunc,_ = updatewage(w,theta=theta,N=N,L=L,T=T,tau=tau,psi=psi)
print(w_newfunc)
print(w_new)

[0.25353982 0.24882006 0.24882006]
[0.25353982 0.24882006 0.24882006]


# While-loopの導入

ここではwhile loopを用いて、賃金が超過需要を十分に小さくするまで、更新を続けていくようなコードを書いていく。ここで閾値tolを定義する。

In [130]:
tol = 0.0001
iter = 1

Z = np.ones((N))
w = np.ones((N))
while max(np.abs(Z)) > tol and iter < 100:
  iter += 1
  w_old = np.copy(w)
  w,Z,_,_ = updatewage(w,theta=4,sigma=3,N=3,L=L,T=T,tau=tau,psi=0.1)
  if iter % 10 == 0:
    print(iter)
    print(Z)
    print(w)

print("Done?")

w,Z,P,X = updatewage(w,theta=4,sigma=3,N=3,L=L,T=T,tau=tau,psi=0.1)
print(Z)
print(w)


10
[ 0.00559218 -0.00294981 -0.00294981]
[0.26031388 0.24656204 0.24656204]
20
[ 1.13421104e-04 -5.99655974e-05 -5.99655974e-05]
[0.26061275 0.24646242 0.24646242]
Done?
[ 5.20614440e-05 -2.75255293e-05 -2.75255293e-05]
[0.26061611 0.2464613  0.2464613 ]


最後に厚生を計算します。厚生は
$$ U_n = \frac{w_n}{P_n} $$
となります。

In [131]:
U = w / P
print(U)

[1.04077555 1.01577716 1.01577716]
